In [ ]:
# =============================================================================
# Lais Aparecida Borges
# Projeto: Corretor de Redações para o ENEM com LLM
#
# Disciplina: CA016IC - Tópicos em Inteligência Computacional
#
# Problema
#
# O ENEM, principal porta de entrada para o ensino superior no Brasil, avalia os
# candidatos em diversas competências. A qualidade e a adequação da redação
# são fatores determinantes para o sucesso dos estudantes.
# No entanto, o acesso a feedback personalizado e detalhado sobre redações pode
# ser limitado, especialmente fora de cursos preparatórios.
#
# Objetivo
#
# Este projeto visa desenvolver um protótipo de corretor de redações para o ENEM
# que integra a funcionalidade de Retrieval-Augmented Generation (RAG) com um
# LLM finetunado para o contexto da língua portuguesa e o exame. O sistema
# receberá um texto de redação e o tema proposto, e utilizará um banco de dados
# vetorial contendo comentários de redações anteriores para gerar um feedback
# detalhado e contextualizado.
#
# O foco principal é na aplicabilidade prática e na geração de feedback que
# aborde os critérios de avaliação do ENEM, utilizando um LLM
# (Qwen2.5-0.5B-PT-BR-Instruct) com adaptações via LoRA e um modelo de embedding
# otimizado para português (paraphrase-multilingual-MiniLM-L12-v2).
#
# Escopo e Limitações
#
# - O corpus de redações utilizado é proveniente do repositório uol-redacoes-xml.
# - A tokenização é realizada pelo tokenizer do modelo Qwen2.5.
# - O modelo base é o Qwen2.5-0.5B-PT-BR-Instruct, finetunado com LoRA.
# - O RAG é implementado utilizando ChromaDB e Sentence Transformers.
# - A avaliação de saída é realizada com métricas ROUGE, BERTScore e BLEU.
# - O feedback gerado é um protótipo e pode necessitar de refinamentos para
#   cobrir todas as nuances de cada competência do ENEM.
#
# Referências
#
# uol-redacoes-xml: Repositório de dados de redações.
# Disponível em: https://github.com/gpassero/uol-redacoes-xml
#
# Hugging Face Transformers, PEFT, Datasets, Evaluate, Sentence Transformers.
# =============================================================================

In [ ]:
# =============================================================================
# PASSO 1: INSTALAÇÃO DE BIBLIOTECAS E PACOTES
# =============================================================================

# Instalação do pacote proveniente do repositório que contém um banco de
# redações em formato XML, com dados como o tema da redação, o texto produzido,
# o texto corrigido, os comentários dos avaliadores, entre outras informações.
!pip install git+https://github.com/gpassero/uol-redacoes-xml.git

# Instalação das bibliotecas necessárias
!pip install -q transformers datasets accelerate peft bitsandbytes trl evaluate chromadb sentence-transformers rouge_score bert_score

  Cloning https://github.com/gpassero/uol-redacoes-xml.git to /tmp/pip-req-build-7ce2on77
  Running command git clone --filter=blob:none --quiet https://github.com/gpassero/uol-redacoes-xml.git /tmp/pip-req-build-7ce2on77
  Resolved https://github.com/gpassero/uol-redacoes-xml.git to commit 94b74fc91c4e7a6b582ebc3708aa0dca2ba12ca6
  Preparing metadata (setup.py) ... done
  Created wheel for uol_redacoes_xml: filename=uol_redacoes_xml-0.2-py3-none-any.whl size=2978835 sha256=48f4b4be65d67531391c5f863e3ebcc73438c21c62c8f38f58e85271a3d6d387
  Stored in directory: /tmp/pip-ephem-wheel-cache-xk782ri6/wheels/c1/e3/ee/70fe667b172b519fa5f401241e6af9b31ab33b05cf715341e5
Successfully built uol_redacoes_xml
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# =============================================================================
# PASSO 2: IMPORTAÇÕES NECESSÁRIAS
# =============================================================================
import pandas as pd
import numpy as np
import torch
import chromadb
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from evaluate import load
from sentence_transformers import SentenceTransformer
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from evaluate import load
from bert_score import score

# Bibliotecas necessárias para a base de dados
import nltk
nltk.download('punkt_tab')
import uol_redacoes_xml

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# =============================================================================
# PASSO 3: CRIAÇÃO DO DATAFRAME
# =============================================================================

# Para evitar erros com o pacote de redações, as informações julgadas
# relevantes para o feedback (tema, redação e comentários) são passadas para um
# DataFrame, que é salvo localmente.

# Carregamento das redações
essays = uol_redacoes_xml.load()

data = []

for essay in essays:
  # Limpeza de tags HTML/XML nos comentários
  essay.comments = essay.comments.replace('<comments>', '')
  essay.comments = essay.comments.replace('</comments>', '')

  data.append({
      'THEME': essay.prompt.title,
      'ESSAY': essay.text,
      'COMMENTS': essay.comments
  })

filename = 'Essays.csv'
df_essays = pd.DataFrame(data)
df_essays.drop_duplicates(subset='ESSAY', inplace=True)
df_essays.dropna(inplace=True)
df_essays.to_csv(filename, index=False)

In [ ]:
# =============================================================================
# PASSO 4: CARREGAMENTO DO MODELO PRÉ-TREINADO
# =============================================================================
model_id = "amadeusai/AV-FI-Qwen2.5-0.5B-PT-BR-Instruct" # Especializado na língua portuguesa

# Tokenização do modelo
tok = AutoTokenizer.from_pretrained(model_id)
if tok.pad_token is None: tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="auto",
    trust_remote_code=True
)
model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/246 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
# =============================================================================
# PASSO 5: CONFIGURAÇÃO DO LORA
# =============================================================================
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

model = get_peft_model(model, lora_cfg)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
      

In [ ]:
# =============================================================================
# PASSO 6: CARREGAMENTO DO MODELO DE EMBEDDINGS
# =============================================================================
# Carregamento do modelo de embedding capaz de compreender
# a intenção e o contexto em português.
embedding_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embedding_model = SentenceTransformer(embedding_model_id)
embedding_model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# =============================================================================
# PASSO 7: PREPARAÇÃO DO BANCO DE DADOS VETORIAL
# =============================================================================
# Configuração do ChromaDB
chroma_client = chromadb.Client()

# Criação da coleção
collection_name = "essays_feedback"

try:
  collection = chroma_client.get_collection(name=collection_name)
  print(f"Collection '{collection_name}' already exists.")
except:
  collection = chroma_client.create_collection(name=collection_name)
  print(f"Collection '{collection_name}' created.")

Collection 'essays_feedback' created.


In [ ]:
# Função para gerar embeddings e adicionar ao ChromaDB
def add_to_chroma(documents: list[str], metadatas: list[dict], ids: list[str]):
  """
  Adiciona documentos ao ChromaDB.

  Args:
    documents (list[str]): Lista dos documentos para adicionar.
    metadatas (list[dict]): Lista dos metadados associados aos documentos.
    ids (list[str]): Lista dos IDs associados aos documentos.

  Returns:
    None
  """
  # Geração dos embeddings
  embeddings = embedding_model.encode(documents, batch_size=32, show_progress_bar=True).tolist()

  # Adição dos embeddings ao ChromaDB
  print(f"\nAdding {len(documents)} documents to ChromaDB...")
  collection.add(
      embeddings=embeddings,
      documents=documents,
      metadatas=metadatas,
      ids=ids
  )
  print(f"{len(documents)} documents added to collection '{collection_name}'")

# Preparação dos dados
docs_to_index = df_essays['COMMENTS'].tolist()
metadatas_to_index = [{"type": "essay_comment"}] * len(docs_to_index)
ids_to_index = [f"essay_comment_{i}" for i in range(len(df_essays))]

add_to_chroma(docs_to_index, metadatas_to_index, ids_to_index)

Batches:   0%|          | 0/68 [00:00<?, ?it/s]


Adding 2162 documents to ChromaDB...
2162 documents added to collection 'essays_feedback'


In [ ]:
# Função para buscar informações relevantes
def search_chroma(query_text: str, n_results: int) -> list[str]:
  """
  Busca os documentos mais relevantes.

  Args:
    query_text (str): Texto de consulta.
    n_results (int): Número de resultados a serem retornados.

  Returns:
    list[str]: Lista dos documentos mais relevantes.
  """
  # Geração do embedding da consulta
  query_embedding = embedding_model.encode(query_text).tolist()

  # Consulta ao ChromaDB para encontrar documentos mais relevantes com base na similaridade dos embeddings
  results = collection.query(
      query_embeddings=[query_embedding],
      n_results=n_results,
      include=['documents', 'metadatas']
  )

  relevant_docs = []
  if results and results.get('documents') and results['documents'][0]:
    for doc, meta in zip(results['documents'][0], results.get('metadatas', [{}])[0]):
      relevant_docs.append({"text": doc, "metadata": meta})

  return relevant_docs

In [ ]:
# =============================================================================
# PASSO 8: PREPARAÇÃO DOS DADOS
# =============================================================================
# Template do prompt para treinamento
TRAINING_IN_FMT = """Você é um assistente especializado em dar feedback para redações do ENEM.

Tema: {tema}

Redação:
{redacao}

Feedback:
"""

# Template do prompt para inferência
RAG_IN_FMT = """Você é um assistente especializado em dar feedback para redações do ENEM.
Utilize os seguintes contextos para auxiliar na geração do feedback, focando em gramática, coesão, coerência e argumentação.
Seja construtivo e objetivo.

{context}

Tema: {tema}

Redação:
{redacao}

Feedback:
"""

# Conversão do DataFrame para um Hugging Face Dataset
ds = Dataset.from_pandas(df_essays)
ds

Dataset({
    features: ['THEME', 'ESSAY', 'COMMENTS', '__index_level_0__'],
    num_rows: 2162
})

In [ ]:
# =============================================================================
# PASSO 9: TOKENIZAÇÃO
# =============================================================================
# Tamanho máximo da sequência
MAX_LEN = 512

# Função de tokenização que processa cada exemplo
def tok_example(example):
  """
  Tokeniza um exemplo de dados.

  Args:
    example (dict): Exemplo de dados.

  Returns:
    dict: Exemplo de dados tokenizados.
  """
  prompt = TRAINING_IN_FMT.format(tema=example['THEME'], redacao=example['ESSAY'])

  tp = tok(prompt, add_special_tokens=False)
  tr = tok(example['COMMENTS'] + tok.eos_token, add_special_tokens=False)

  ids = (tp['input_ids'] + tr['input_ids'])[:MAX_LEN]
  att = [1] * len(ids)
  lab = ([-100]*len(tp["input_ids"]) + tr["input_ids"])[:MAX_LEN]

  return {"input_ids": ids, "attention_mask": att, "labels": lab}

train_ds = ds.map(tok_example, remove_columns=ds.column_names)
train_ds

Map:   0%|          | 0/2162 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2162
})

In [ ]:
# =============================================================================
# PASSO 10: BATCH
# =============================================================================
def collate(batch):
  """
  Função de processamento de lote.

  Args:
    batch (list): Lista de exemplos de dados.

  Returns:
    dict: Exemplo de dados processados.
  """

  # Função auxiliar para a organização das redações
  def pad_list(lst, pad_value):
    maxlen = max(len(x) for x in lst)

    out = []
    for x in lst:
      t = torch.tensor(x, dtype=torch.long)

      if t.size(0) < maxlen:
        t = torch.nn.functional.pad(t, (0, maxlen - t.size(0)), value=pad_value)
      out.append(t)

    return torch.stack(out)

  ids = [b["input_ids"] for b in batch]
  att = [b["attention_mask"] for b in batch]
  lab = [b["labels"] for b in batch]

  return{
    "input_ids": pad_list(ids, tok.pad_token_id),
    "attention_mask": pad_list(att, 0),
    "labels": pad_list(lab, -100)
  }

In [ ]:
# =============================================================================
# PASSO 11: CONFIGURAÇÃO DO TREINAMENTO
# =============================================================================
# Criação de um batch com 12 exemplos para ajudar no aprendizado do modelo
# de uma maneira mais estável com per_device_train_batch_size=4 e
# gradient_accumulation_steps=3.

# Aumento gradual da velocidade de aprendizado no início e, depois, diminuição
# suave com lr_scheduler_type="cosine" e warmup_steps=50.

args = TrainingArguments(
    output_dir="./essay",
    max_steps=300, # Controle da duração do treino
    per_device_train_batch_size=4,
    gradient_accumulation_steps=3,
    learning_rate=2e-5, # Velocidade de aprendizado mais segura
    bf16=True, # Treino mais rápido, mas com um menor uso da memória
    group_by_length=True, # Organização dos exemplos de redações por tamanho
    lr_scheduler_type="cosine",
    warmup_steps=50,
    logging_steps=50,
    save_steps=100,
    report_to="none",
    optim="paged_adamw_32bit"
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collate)
trainer.model.config

The model is already on multiple devices. Skipping the move to device specified in `args`.


Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_

In [ ]:
# =============================================================================
# PASSO 12: TREINAMENTO
# =============================================================================
trainer.train()

Step,Training Loss
50,2.784600


In [ ]:
# =============================================================================
# PASSO 13: GERAÇÃO
# =============================================================================
# Antes da busca, a redação é truncada para prevenção de erros de dimensão no
# modelo de embedding, focando no início/corpo do texto, o que garante que o
# contexto recuperado seja semanticamente relevante.
def generate(instr):
  """
  Função para gerar feedback a partir de uma redação.

  Args:
    instr (dict): Dicionário contendo o tema e a redação.

  Returns:
    generated_text (str): Feedback gerado.
  """
  # Busca pelos trechos de texto mais relevantes relacionados à redação em questão
  max_query_tokens = 256 # Limite para a consulta de busca
  tokens_obj = tok(instr['ESSAY'], max_length=max_query_tokens, truncation=True, return_tensors="pt", add_special_tokens=False)
  search_query = tok.decode(tokens_obj['input_ids'][0], skip_special_tokens=True)
  relevant_contexts = search_chroma(search_query, 3) # 3 trechos mais relevantes

  instr_context = "\n".join([doc["text"] for doc in relevant_contexts])
  prompt = RAG_IN_FMT.format(context=instr_context, tema=instr['THEME'], redacao=instr['ESSAY'])

  x = tok(prompt, return_tensors="pt").to(model.device)

  # Geração da resposta
  with torch.no_grad():
    y = model.generate(
        **x,
        max_new_tokens=600,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
        no_repeat_ngram_size=5 # Força a diversidade vocabular, impedindo loops de frase comuns
    )

    # Processamento da saída
    input_len = x["input_ids"].shape[1]
    generated_tokens = y[0][input_len:]
    generated_text = tok.decode(generated_tokens, skip_special_tokens=True)

    print("="*40)
    print(f"TEMA: {instr['THEME']}")
    print("-" * 40)
    print("FEEDBACK:\n")
    print(generated_text)
    print("="*40)

    return generated_text

In [ ]:
# Teste
example = df_essays.iloc[1]
generated_text = generate(example)

In [ ]:
# =============================================================================
# PASSO 14: AVALIAÇÃO
# =============================================================================
rouge = load("rouge")

def evaluate(reference_text, generated_text, lang="pt"):
  """
  Função para avaliar a saída do modelo.

  Args:
    reference_text (str): Texto de referência.
    generated_text (str): Texto gerado.
    lang (str): Idioma dos textos.

  Returns:
    rouge_results (dict): Resultados do ROUGE.
    bert_precision (torch.Tensor): Precisão do BERT.
    bert_recall (torch.Tensor): Revocação do BERT.
    bert_f1 (torch.Tensor): F1
    bleu_score (float): BLEU.
  """
  references = [reference_text]
  predictions = [generated_text]

  # ROUGE
  rouge_results = rouge.compute(predictions=predictions, references=references)
  # BERTScore
  bert_precision, bert_recall, bert_f1 = score(references, predictions, lang=lang, verbose=False)
  # BLEU
  bleu_score = sentence_bleu(references, generated_text)

  return rouge_results, bert_precision, bert_recall, bert_f1, bleu_score

rouge_results, bert_precision, bert_recall, bert_f1, bleu_score = evaluate(example['COMMENTS'], generated_text)

print("="*40)
print("   AVALIAÇÃO")
print("ROUGE Scores:")
print(f" ROUGE-1: {rouge_results['rouge1']:.3f}")
print(f" ROUGE-2: {rouge_results['rouge2']:.3f}")
print(f" ROUGE-L: {rouge_results['rougeL']:.3f}")
print("-"*40)
print("BERT Scores:")
print(f" Precision: {bert_precision.mean().item():.3f}")
print(f" Recall:    {bert_recall.mean().item():.3f}")
print(f" F1:        {bert_f1.mean().item():.3f}")
print("-"*40)
print(f"BLEU Score: {bleu_score:.3f}")
print("="*40)

***(ALTERAR!)***

Os resultados indicam um modelo que aprendeu o "conceito" do feedback, mas manteve liberdade criativa na escrita:

* Sucesso Semântico (BERTScore ~0.74): Este é o indicador mais crítico. Um F1 de 0.75 é considerado alto para tarefas de geração aberta. Isso prova que, mesmo usando palavras diferentes, o modelo preservou a intenção e o significado da referência. O Recall (0.747) superior à Precision sugere que o modelo é exaustivo: ele cobre quase todos os pontos da referência, raramente omitindo críticas importantes.

* Comportamento Abstrativo (ROUGE Gap): A queda acentuada entre ROUGE-1 (51% de overlap de palavras) e ROUGE-L (16% de estrutura frasal) é positiva neste contexto. Ela confirma que o modelo não está memorizando (overfitting) frases inteiras. Ele absorveu o vocabulário técnico (garantindo o ROUGE-1 e o BLEU alto de 0.61), mas constrói sentenças novas sintaticamente distintas da referência.